In [ ]:
# initialize stuff
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import importlib
import logging
import pickle

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.special import jv
from scipy.interpolate import interp1d
from numpy.typing import NDArray

sys.path.append("/Users/oliver/Documents/p5control-bluefors-evaluation")
sys.path.append("/home/oliver/Documents/p5control-bluefors-evaluation")

from utilities.corporate_design_colors_v4 import cmap
importlib.reload(sys.modules["utilities.corporate_design_colors_v4"])

from theory.dynes.dynes import get_current_dynes
importlib.reload(sys.modules["theory.dynes.dynes"])

from theory.carlosha.ha import get_current_ha
importlib.reload(sys.modules["theory.carlosha.ha"])

from theory.carlosfcs.fcs import get_current_fcs
importlib.reload(sys.modules["theory.carlosfcs.fcs"])

from theory.tien_gordon.tien_gordon import get_tien_gordon_pat
from theory.tien_gordon.tien_gordon import get_tien_gordon_pamar
importlib.reload(sys.modules["theory.tien_gordon.tien_gordon"])

import utilities.basefunctions
importlib.reload(sys.modules["utilities.basefunctions"])

from scipy.constants import e, h
G_0 = 2 * e**2 / h


In [4]:
import numpy as np

# Constants
pi = np.pi
gamma_euler = 0.5772156649
e_charge = 1.602e-19  # elementary charge (C)
hbar = 1.055e-34      # reduced Planck constant (J·s)
G0 = 2 * e_charge**2 / hbar  # Quantum of conductance (S)

def bcs_gap(temp_reduced):
    if temp_reduced >= 1.0:
        return 0.0
    if temp_reduced == 0.0:
        return 1.0
    if temp_reduced > 0.99:
        return np.exp(gamma_euler) * np.sqrt(8 * (1 - temp_reduced) / (7 * 1.202))
    
    def integrand(x, gapt, tempf):
        w = gapt * np.sqrt(x**2 + 1) * pi * np.exp(-gamma_euler)
        if w / tempf < 50:
            fermi = 1.0 / (1.0 + np.exp(w / tempf))
        else:
            fermi = 0.0
        return -2.0 * fermi / np.sqrt(x**2 + 1)
    
    gapt = 1.0
    tempf = temp_reduced * pi / np.exp(gamma_euler)
    for _ in range(100):
        x_max = 50.0 * tempf / (gapt * pi * np.exp(-gamma_euler))
        x_vals = np.linspace(0, x_max, 10000)
        dx = x_vals[1] - x_vals[0]
        integral = np.sum([integrand(x, gapt, tempf) for x in x_vals]) * dx
        gaptf = np.exp(integral)
        if np.abs(gaptf - gapt) < 1e-5:
            return gaptf
        gapt = gaptf
    return gapt

In [9]:
bcs_gap(.9)

np.float64(0.0)